In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [16]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[0.0918, 0.0992, 0.1001, 0.1048, 0.0947, 0.1093, 0.0999, 0.1000, 0.1031,
         0.0973]], grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([5])


In [17]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [18]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [22]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [23]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2956,  0.0776, -0.1639, -0.1913,  0.0630, -0.0370,  0.3193,  0.2023,
          0.3862,  0.2692, -0.3129, -0.7001, -1.0598,  0.3199, -0.3901,  0.1385,
          0.5549, -0.2340,  0.7443,  0.3015],
        [-0.2138,  0.0172, -0.2833, -0.5999,  0.0506, -0.3679,  0.4734,  0.2680,
          0.3124,  0.1634, -0.5794, -0.3325, -0.8200,  0.5228, -0.4754,  0.2975,
          0.2297, -0.2481,  0.4769,  0.2414],
        [-0.0631,  0.1418, -0.2812, -0.1509,  0.2019, -0.1649,  0.6526,  0.1290,
          0.4069,  0.1675, -0.5847, -0.1102, -0.6738,  0.5974, -0.5171,  0.0170,
          0.4742, -0.2704,  0.4936,  0.2755]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0776, 0.0000, 0.0000, 0.0630, 0.0000, 0.3193, 0.2023, 0.3862,
         0.2692, 0.0000, 0.0000, 0.0000, 0.3199, 0.0000, 0.1385, 0.5549, 0.0000,
         0.7443, 0.3015],
        [0.0000, 0.0172, 0.0000, 0.0000, 0.0506, 0.0000, 0.4734, 0.2680, 0.3124,
         0.1634, 0.0000, 0.0000, 0.0000, 0.5228, 0.00

In [24]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [25]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [26]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0279, -0.0190,  0.0159,  ...,  0.0013,  0.0283,  0.0145],
        [ 0.0290, -0.0033,  0.0285,  ..., -0.0196, -0.0020, -0.0119]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0093, 0.0115], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0361,  0.0281, -0.0130,  ...,  0.0111,  0.0330,  0.0002],
        [-0.0286, -0.0155, -0.0167,  ...,  0.0243, -0.0097, -0.0240]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si